In [1]:
import pandas as pd
import os

Lo que haremos aquí es recopilar los datos de las reviews de los estados que utilizaremos en el análisis. Los archivos en crudo se presentaron en formato Json, en archivos individuales para cada uno de los 51 estados de EEUU, por lo que procederemos a ordenar los mismos, en formato parquet, y por la dimension, en dataframes individuales, para luego crear una tabla única para los 5 estados pertinentes.

# New York

Se convierten los archivos Json a formato parquet

In [6]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE\review-New_York"
# Carpeta de salida para guardar los archivos Parquet
output_folder =r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet"

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet\1.parquet
Convertido: 10.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet\10.parquet
Convertido: 11.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet\11.parquet
Convertido: 12.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet\12.parquet
Convertido: 13.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet\13.parquet
Convertido: 14.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet\14.parquet
Convertido: 15.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet\15.parquet
Convertido: 16.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Re

In [7]:
# Leer el archivo Parquet
Rev_New_York_1 = pd.read_parquet(r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet")

print(Rev_New_York_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB
None


Se concatenan los archivos parquet para formar un dataframe

In [8]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-New_York.parquet"

# Lista para almacenar DataFrames
df_list_New_York = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_ny = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_ny)
        df_list_New_York.append(df)

# Concatenar todos los DataFrames en uno solo
df_new_york = pd.concat(df_list_New_York, ignore_index=True)

# Verificar el DataFrame combinado
print(df_new_york.shape)
print

(2700000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [9]:
df_new_york = df_new_york.drop(columns=['pics'])

Se convierten las columnas necesarias

In [10]:
# Se convierte la columna 'user_id' a tipo object
df_new_york['user_id'] = df_new_york['user_id'].astype('str')
print(df_new_york['user_id'].head(3))

0    1.1372210469230823e+20
1    1.0729344149210933e+20
2     1.003785858018194e+20
Name: user_id, dtype: object


In [11]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_new_york['time'] = pd.to_datetime(df_new_york['time'], unit='ms').dt.normalize()
print(df_new_york['time'].head(3))

0   2020-10-23
1   2021-05-04
2   2020-08-14
Name: time, dtype: datetime64[ns]


In [12]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_ny = pd.json_normalize(df_new_york['resp'])
df_new_york[['resp_text', 'resp_time']] = resp_df_ny[['text', 'time']]

In [13]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_new_york['resp_time'] = pd.to_datetime(df_new_york['resp_time'], unit='ms').dt.normalize()
print(df_new_york['resp_time'].head(3))

0          NaT
1   2021-05-06
2          NaT
Name: resp_time, dtype: datetime64[ns]


In [14]:
# Se elimina la columna 'resp' original
df_new_york = df_new_york.drop(columns=['resp'])

In [15]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_new_york = df_new_york.rename(columns={'gmap_id': 'id_negocio'})

In [16]:
# Se reordena la columna 'id_negocio'
columnas = list(df_new_york.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_new_york = df_new_york[columnas]

In [17]:
# Se añade la columna 'review_id' autoincremental
df_new_york.insert(0, 'review_id', range(1, 1 + len(df_new_york)))

In [18]:
# Se añade la columna 'Estado'
df_new_york.insert(2, 'estado', 'New York')

In [19]:
# Se verifica como queda el dataframe
print(df_new_york.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 206.0+ MB
None


In [20]:
df_new_york

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,1,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.1372210469230823e+20,Alvin Martinez,2020-10-23,5,I'm late to posting this but this store especi...,NaN,NaT
1,2,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.0729344149210933e+20,Johnnie Jackson,2021-05-04,1,Very dissatisfied I did not get my phone the p...,We pride ourselves on providing an awesome exp...,2021-05-06
2,3,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.003785858018194e+20,Manie Blazer,2020-08-14,5,Excellent very well done with professional car...,NaN,NaT
3,4,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.1499816115301982e+20,Fashion Fiinds,2018-12-02,5,Basing my review strictly on the service I rec...,Thanks for the awesome review! We work hard t...,2018-12-03
4,5,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.171781857284223e+20,Andres Rieloff,2020-08-13,1,Bad! Disorganized. I'm being totally honest. I...,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
2699995,2699996,0x89de0b8a0905153d:0x976fc4a006084f03,New York,1.1508884569871743e+20,Gourav Saha,2019-04-16,4,None,NaN,NaT
2699996,2699997,0x89de0b8a0905153d:0x976fc4a006084f03,New York,1.1170639723279111e+20,jenn mosher,2016-05-11,5,None,NaN,NaT
2699997,2699998,0x89de0b8a0905153d:0x976fc4a006084f03,New York,1.0836978407597677e+20,Michele Huck,2019-11-16,5,None,NaN,NaT
2699998,2699999,0x89de0b8a0905153d:0x976fc4a006084f03,New York,1.0078678731635175e+20,Frank B,2020-10-27,5,None,NaN,NaT


# California

Se pasan de formato Json a Parquet

In [21]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE\review-California"
# Carpeta de salida para guardar los archivos Parquet
output_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet"

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet\1.parquet
Convertido: 10.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet\10.parquet
Convertido: 11.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet\11.parquet
Convertido: 12.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet\12.parquet
Convertido: 13.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet\13.parquet
Convertido: 14.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet\14.parquet
Convertido: 15.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet\15.parquet
Convertido: 16.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-

In [22]:
# Leer el archivo Parquet
Rev_California_1 = pd.read_parquet(r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet")

# Mostrar las primeras filas
print(Rev_California_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB
None


Se concatenan los archivos .parquet para formar un Dataframe

In [23]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\OESTE.parquet\Rev-California.parquet"

# Lista para almacenar DataFrames
df_list_California = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_nmex = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_nmex)
        df_list_California.append(df)

# Concatenar todos los DataFrames en uno solo
df_california = pd.concat(df_list_California, ignore_index=True)

# Verificar el DataFrame combinado
print(df_california.shape)
print

(2700000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [24]:
df_california = df_california.drop(columns=['pics'])

Se convierten las columnas necesarias

In [25]:
# Se convierte la columna 'user_id' a tipo object
df_california['user_id'] = df_california['user_id'].astype('str')
print(df_california['user_id'].head(3))


0    1.0899115226265579e+20
1    1.1129032221979622e+20
2    1.1264035744961195e+20
Name: user_id, dtype: object


In [26]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_california['time'] = pd.to_datetime(df_california['time'], unit='ms').dt.normalize()
print(df_california['time'].head(3))

0   2021-01-06
1   2021-02-09
2   2020-03-08
Name: time, dtype: datetime64[ns]


In [27]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_cal = pd.json_normalize(df_california['resp'])
df_california[['resp_text', 'resp_time']] = resp_df_cal[['text', 'time']]

In [28]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_california['resp_time'] = pd.to_datetime(df_california['resp_time'], unit='ms').dt.normalize()
print(df_california['resp_time'].head(3))

0   NaT
1   NaT
2   NaT
Name: resp_time, dtype: datetime64[ns]


In [29]:
# Se elimina la columna 'resp' original
df_california = df_california.drop(columns=['resp'])

In [30]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_california = df_california.rename(columns={'gmap_id': 'id_negocio'})

In [31]:
# Se reordena la columna 'id_negocio'
columnas = list(df_california.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_california = df_california[columnas]

In [32]:
# Se añade la columna 'review_id' autoincremental desde 2700001 para continuar desde el df_new_york
df_california.insert(0, 'review_id', range(2700001, 2700001 + len(df_california)))

In [33]:
# Se añade la columna 'Estado'
df_california.insert(2, 'estado', 'California')

In [34]:
# Se verifica como queda el dataframe
print(df_california.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 206.0+ MB
None


In [35]:
df_california

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,2700001,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California,1.0899115226265579e+20,Song Ro,2021-01-06,5,Love there korean rice cake.,NaN,NaT
1,2700002,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California,1.1129032221979622e+20,Rafa Robles,2021-02-09,5,Good very good,NaN,NaT
2,2700003,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California,1.1264035744961195e+20,David Han,2020-03-08,4,They make Korean traditional food very properly.,NaN,NaT
3,2700004,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California,1.1744034972382365e+20,Anthony Kim,2019-03-07,5,Short ribs are very delicious.,NaN,NaT
4,2700005,0x80c2c778e3b73d33:0xbdc58662a4a97d49,California,1.0058077083612353e+20,Mario Marzouk,2017-05-16,5,Great food and prices the portions are large,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
2699995,5399996,0x80c2bea30829f279:0x39aa953ee93734ed,California,1.1134233708701893e+20,Byunguk Kim,2019-11-10,5,None,NaN,NaT
2699996,5399997,0x8094675073616747:0x9f935a9b9046a9ba,California,1.080818202512737e+20,Ceveda Craytonhooks,2020-02-02,5,Beautiful salon. The staff and atmosphere were...,NaN,NaT
2699997,5399998,0x8094675073616747:0x9f935a9b9046a9ba,California,1.1759138313533325e+20,Natacha Thompson,2018-06-10,5,Flo is a great beautician. She is very patie...,Thank you! It was my pleasure to give you exa...,2018-06-10
2699998,5399999,0x8094675073616747:0x9f935a9b9046a9ba,California,1.1242661065579289e+20,Eleanor Aikins,2019-02-26,5,I Been going to have a variety of hair style a...,NaN,NaT


# Pennsylvania

Se convierten los archivos Json a formato parquet

In [36]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE\review-Pennsylvania"
# Carpeta de salida para guardar los archivos Parquet
output_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet"

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet\1.parquet
Convertido: 10.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet\10.parquet
Convertido: 11.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet\11.parquet
Convertido: 12.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet\12.parquet
Convertido: 13.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet\13.parquet
Convertido: 14.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet\14.parquet
Convertido: 15.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet\15.parquet
Convertido: 16.json a C:\Users\Juli\Desktop\Data Science\Archivos Proye

In [37]:
# Leer el archivo Parquet
Rev_Pennsylvania_1 = pd.read_parquet(r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet")

print(Rev_Pennsylvania_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 146.5+ MB
None


Se concatenan los archivos parquet para formar un solo dataframe

In [38]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\NORESTE_parquet\Rev-Pennsylvania.parquet"

# Lista para almacenar DataFrames
df_list_Pennsylvania = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_psv = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_psv)
        df_list_Pennsylvania.append(df)

# Concatenar todos los DataFrames en uno solo
df_pennsylvania = pd.concat(df_list_Pennsylvania, ignore_index=True)

# Verificar el DataFrame combinado
print(df_pennsylvania.shape)
print

(2400000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [39]:
df_pennsylvania = df_pennsylvania.drop(columns=['pics'])

Se convierten las columnas necesarias

In [40]:
# Se convierte la columna 'user_id' a tipo object
df_pennsylvania['user_id'] = df_pennsylvania['user_id'].astype('str')
print(df_pennsylvania['user_id'].head(3))

0    1.0494474255907976e+20
1    1.1760970283298033e+20
2    1.1056324201842663e+20
Name: user_id, dtype: object


In [41]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_pennsylvania['time'] = pd.to_datetime(df_pennsylvania['time'], unit='ms').dt.normalize()
print(df_pennsylvania['time'].head(3))

0   2018-02-04
1   2016-10-12
2   2012-06-04
Name: time, dtype: datetime64[ns]


In [42]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_pen = pd.json_normalize(df_pennsylvania['resp'])
df_pennsylvania[['resp_text', 'resp_time']] = resp_df_pen[['text', 'time']]

In [43]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_pennsylvania['resp_time'] = pd.to_datetime(df_pennsylvania['resp_time'], unit='ms').dt.normalize()
print(df_pennsylvania['resp_time'].head(3))

0   NaT
1   NaT
2   NaT
Name: resp_time, dtype: datetime64[ns]


In [44]:
# Se elimina la columna 'resp' original
df_pennsylvania = df_pennsylvania.drop(columns=['resp'])

In [45]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_pennsylvania = df_pennsylvania.rename(columns={'gmap_id': 'id_negocio'})

In [46]:
# Se reordena la columna 'id_negocio'
columnas = list(df_pennsylvania.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_pennsylvania = df_pennsylvania[columnas]

In [47]:
# Se añade la columna 'review_id' autoincremental desde 4500001 para continuar desde el df_california
df_pennsylvania.insert(0, 'review_id', range(4500001, 4500001 + len(df_pennsylvania)))

In [48]:
# Se añade la columna 'Estado'
df_pennsylvania.insert(2, 'estado', 'Pennsylvania')

In [49]:
# Se verifica como queda el dataframe
print(df_pennsylvania.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 183.1+ MB
None


In [51]:
df_pennsylvania

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,4500001,0x89c6c63c8cd87141:0x54d0d283872eecbb,Pennsylvania,1.0494474255907976e+20,Jaron Whitfield,2018-02-04,5,"Joe is quite unique of his line of work, he as...",NaN,NaT
1,4500002,0x89c6c63c8cd87141:0x54d0d283872eecbb,Pennsylvania,1.1760970283298033e+20,Jonathan McCarthy,2016-10-12,5,For such a small place their impact on my life...,NaN,NaT
2,4500003,0x89c6c63c8cd87141:0x54d0d283872eecbb,Pennsylvania,1.1056324201842663e+20,Rocky Kev,2012-06-04,5,I usually give them a call before I stop by to...,NaN,NaT
3,4500004,0x89c6c63c8cd87141:0x54d0d283872eecbb,Pennsylvania,1.128959735054014e+20,Josep Valls,2013-03-14,5,My bike had been sitting outdoors for a good w...,NaN,NaT
4,4500005,0x89c6c63c8cd87141:0x54d0d283872eecbb,Pennsylvania,1.1061967488596383e+20,Timaree Schmit,2019-01-29,5,Always an easy experience. Service is knowledg...,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
2399995,6899996,0x89cb5b2714dfeecf:0xe4f25ae9daa774ff,Pennsylvania,1.091213518597228e+20,Aileen Blas,2021-04-05,5,"Great cookies, wonderful staff. Very enjoyabl...",NaN,NaT
2399996,6899997,0x89cb5b2714dfeecf:0xe4f25ae9daa774ff,Pennsylvania,1.0787372814043614e+20,Joshua Juda,2020-10-29,4,Good fresh baked cookies. Prices are a little ...,NaN,NaT
2399997,6899998,0x89cb5b2714dfeecf:0xe4f25ae9daa774ff,Pennsylvania,1.0722323937392198e+20,Anthony Casto,2021-03-01,5,Best cookies ever,NaN,NaT
2399998,6899999,0x89cb5b2714dfeecf:0xe4f25ae9daa774ff,Pennsylvania,1.0584293034572984e+20,Mark McKillop,2021-01-19,5,Great place for a snack. Open really late and ...,NaN,NaT


# Florida

Se convierten los archivos Json a formato parquet

In [52]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR\review-Florida"
# Carpeta de salida para guardar los archivos Parquet
output_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet"

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\1.parquet
Convertido: 10.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\10.parquet
Convertido: 11.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\11.parquet
Convertido: 12.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\12.parquet
Convertido: 13.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\13.parquet
Convertido: 14.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\14.parquet
Convertido: 15.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\15.parquet
Convertido: 16.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet\16.parquet
Convertido

In [53]:
# Leer el archivo Parquet
Rev_Florida_1 = pd.read_parquet(r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet")

print(Rev_Florida_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850000 entries, 0 to 2849999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 174.0+ MB
None


Se concatenan los archivos parquet para formar un solo dataframe

In [54]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Florida.parquet"

# Lista para almacenar DataFrames
df_list_Florida = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_psv = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_psv)
        df_list_Florida.append(df)

# Concatenar todos los DataFrames en uno solo
df_florida = pd.concat(df_list_Florida, ignore_index=True)

# Verificar el DataFrame combinado
print(df_florida.shape)
print

(2850000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [55]:
df_florida = df_florida.drop(columns=['pics'])

Se convierten las columnas necesarias

In [56]:
# Se convierte la columna 'user_id' a tipo object
df_florida['user_id'] = df_florida['user_id'].astype('str')
print(df_florida['user_id'].head(3))

0    1.0147185615514873e+20
1    1.1547723478903833e+20
2    1.0180501024489284e+20
Name: user_id, dtype: object


In [57]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_florida['time'] = pd.to_datetime(df_florida['time'], unit='ms').dt.normalize()
print(df_florida['time'].head(3))

0   2021-08-03
1   2020-07-18
2   2018-04-05
Name: time, dtype: datetime64[ns]


In [58]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_fl = pd.json_normalize(df_florida['resp'])
df_florida[['resp_text', 'resp_time']] = resp_df_fl[['text', 'time']]

In [59]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_florida['resp_time'] = pd.to_datetime(df_florida['resp_time'], unit='ms').dt.normalize()
print(df_florida['resp_time'].head(3))

0   2021-07-23
1   2020-02-23
2          NaT
Name: resp_time, dtype: datetime64[ns]


In [60]:
# Se elimina la columna 'resp' original
df_florida = df_florida.drop(columns=['resp'])

In [61]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_florida = df_florida.rename(columns={'gmap_id': 'id_negocio'})

In [62]:
# Se reordena la columna 'id_negocio'
columnas = list(df_florida.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_florida = df_florida[columnas]

In [63]:
# Se añade la columna 'review_id' autoincremental desde 6900001 para continuar desde el df_pennsylvania
df_florida.insert(0, 'review_id', range(6900001, 6900001 + len(df_florida)))

In [64]:
# Se añade la columna 'estado'
df_florida.insert(2, 'estado', 'Florida')

In [65]:
# Se verifica como queda el dataframe
print(df_florida.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850000 entries, 0 to 2849999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 217.4+ MB
None


In [66]:
df_florida.tail()

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
2849995,9749996,0x8890966585e36d3f:0x131d47c2c60a8d31,Florida,1.0028771480080918e+20,James Rudolph,2021-04-01,5,None,Thank you!,2021-04-04
2849996,9749997,0x8890966585e36d3f:0x131d47c2c60a8d31,Florida,1.1409893309742336e+20,Vincent Alexander,2020-02-16,3,None,NaN,NaT
2849997,9749998,0x8890966585e36d3f:0x131d47c2c60a8d31,Florida,1.1234480481217744e+20,Brett Owen,2019-02-27,5,None,NaN,NaT
2849998,9749999,0x8890966585e36d3f:0x131d47c2c60a8d31,Florida,1.0544647183143027e+20,ashly kindle,2018-03-15,5,None,"Thank you, Ashly",2018-01-25
2849999,9750000,0x8890966585e36d3f:0x131d47c2c60a8d31,Florida,1.0874328165743621e+20,Jomarra Auge,2018-09-09,5,None,"Thank you, Mrs. Jomarra:)",2018-09-09


# Texas

Se convierten los archivos Json a formato parquet

In [67]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR\review-Texas"
# Carpeta de salida para guardar los archivos Parquet
output_folder = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet"

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\1.parquet
Convertido: 10.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\10.parquet
Convertido: 11.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\11.parquet
Convertido: 12.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\12.parquet
Convertido: 13.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\13.parquet
Convertido: 14.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\14.parquet
Convertido: 15.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\15.parquet
Convertido: 16.json a C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet\16.parquet
Convertido: 2.json a C:\Us

In [68]:
# Leer el archivo Parquet
Rev_Texas_1 = pd.read_parquet(r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet")

print(Rev_Texas_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296824 entries, 0 to 2296823
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 140.2+ MB
None


Se concatenan los archivos parquet para formar un solo dataframe

In [70]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\SUR_parquet\Rev-Texas.parquet"

# Lista para almacenar DataFrames
df_list_Texas = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_psv = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_psv)
        df_list_Texas.append(df)

# Concatenar todos los DataFrames en uno solo
df_texas = pd.concat(df_list_Texas, ignore_index=True)

# Verificar el DataFrame combinado
print(df_texas.shape)
print

(2296824, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [71]:
df_texas = df_texas.drop(columns=['pics'])

Se convierten las columnas necesarias

In [72]:
# Se convierte la columna 'user_id' a tipo object
df_texas['user_id'] = df_texas['user_id'].astype('str')
print(df_texas['user_id'].head(3))

0    1.1054529907897031e+20
1    1.0361978809750612e+20
2    1.0191666310909164e+20
Name: user_id, dtype: object


In [73]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_texas['time'] = pd.to_datetime(df_texas['time'], unit='ms').dt.normalize()
print(df_texas['time'].head(3))

0   2021-07-04
1   2019-09-13
2   2018-03-01
Name: time, dtype: datetime64[ns]


In [74]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_tx = pd.json_normalize(df_texas['resp'])
df_texas[['resp_text', 'resp_time']] = resp_df_tx[['text', 'time']]

In [75]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_texas['resp_time'] = pd.to_datetime(df_texas['resp_time'], unit='ms').dt.normalize()
print(df_texas['resp_time'].head(3))

0   NaT
1   NaT
2   NaT
Name: resp_time, dtype: datetime64[ns]


In [76]:
# Se elimina la columna 'resp' original
df_texas = df_texas.drop(columns=['resp'])

In [77]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_texas = df_texas.rename(columns={'gmap_id': 'id_negocio'})

In [78]:
# Se reordena la columna 'id_negocio'
columnas = list(df_texas.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_texas = df_texas[columnas]

In [79]:
# Se añade la columna 'review_id' autoincremental desde 9750001 para continuar desde el df_florida
df_texas.insert(0, 'review_id', range(9750001, 9750001 + len(df_texas)))

In [80]:
# Se añade la columna 'estado'
df_texas.insert(2, 'estado', 'Texas')

In [ ]:
# Se verifica como queda el dataframe
print(df_texas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296824 entries, 0 to 2296823
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 175.2+ MB
None


In [82]:
df_texas

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,9750001,0x864c3998b8d8dc83:0x57ffabe1e2322320,Texas,1.1054529907897031e+20,Kimberly Feger,2021-07-04,5,"The pharmacist, Erin, is phenomenal. She was s...",NaN,NaT
1,9750002,0x864c3998b8d8dc83:0x57ffabe1e2322320,Texas,1.0361978809750612e+20,Briana Streit,2019-09-13,2,I gave them 2 stars because they offer prescri...,NaN,NaT
2,9750003,0x864c3998b8d8dc83:0x57ffabe1e2322320,Texas,1.0191666310909164e+20,Sylvia Caudillo,2018-03-01,1,If I could put minus stars I would. This has t...,NaN,NaT
3,9750004,0x864c3998b8d8dc83:0x57ffabe1e2322320,Texas,1.178708983045825e+20,Ginger Kinyon,2019-10-21,1,Please fix your restroom doors,NaN,NaT
4,9750005,0x864c3998b8d8dc83:0x57ffabe1e2322320,Texas,1.1039734611541672e+20,Angeles Arellano,2019-01-04,1,This pharmacy Walmart dose not work not come a...,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
2296819,12046820,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0216813568763861e+20,t williams,2019-07-11,5,None,NaN,NaT
2296820,12046821,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0541730918471213e+20,Erika Lira,2018-07-19,5,None,NaN,NaT
2296821,12046822,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.10785699186536e+20,Jon Brent,2019-09-08,5,None,NaN,NaT
2296822,12046823,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0916760195743087e+20,lunna cabal,2017-09-27,4,None,NaN,NaT


In [99]:
# Se concatenan los  DataFrames anteriores para formar uno solo
g_estados_seleccionados = pd.concat([df_new_york, df_california, df_pennsylvania, df_florida, df_texas])

In [101]:
print(g_estados_seleccionados.info())

<class 'pandas.core.frame.DataFrame'>
Index: 12946824 entries, 0 to 2296823
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 1.1+ GB
None


In [102]:
g_estados_seleccionados

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,1,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.1372210469230823e+20,Alvin Martinez,2020-10-23,5,I'm late to posting this but this store especi...,NaN,NaT
1,2,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.0729344149210933e+20,Johnnie Jackson,2021-05-04,1,Very dissatisfied I did not get my phone the p...,We pride ourselves on providing an awesome exp...,2021-05-06
2,3,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.003785858018194e+20,Manie Blazer,2020-08-14,5,Excellent very well done with professional car...,NaN,NaT
3,4,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.1499816115301982e+20,Fashion Fiinds,2018-12-02,5,Basing my review strictly on the service I rec...,Thanks for the awesome review! We work hard t...,2018-12-03
4,5,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.171781857284223e+20,Andres Rieloff,2020-08-13,1,Bad! Disorganized. I'm being totally honest. I...,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...
2296819,12046820,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0216813568763861e+20,t williams,2019-07-11,5,None,NaN,NaT
2296820,12046821,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0541730918471213e+20,Erika Lira,2018-07-19,5,None,NaN,NaT
2296821,12046822,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.10785699186536e+20,Jon Brent,2019-09-08,5,None,NaN,NaT
2296822,12046823,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0916760195743087e+20,lunna cabal,2017-09-27,4,None,NaN,NaT


In [103]:
nulos_por_columna = g_estados_seleccionados.isnull().sum()
print(nulos_por_columna)

review_id            0
id_negocio           0
estado               0
user_id              0
name                 0
time                 0
rating               0
text           5374662
resp_text     11307310
resp_time     11307310
dtype: int64


Los nulos están en las columnas text, resp_text y resp_time. Para tratarlos voy a reemplazarlos por: 
- nulos en 'text' por "sin comentario"
- nulos en 'resp_text' por "sin respuesta"

In [104]:
# Reemplaza los valores nulos en la columna 'text' con "sin comentario"
g_estados_seleccionados['text'] = g_estados_seleccionados['text'].fillna("sin comentario")

# Reemplaza los valores nulos en la columna 'resp_text' con "sin respuesta"
g_estados_seleccionados['resp_text'] = g_estados_seleccionados['resp_text'].fillna("sin respuesta")

In [105]:
g_estados_seleccionados

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,1,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.1372210469230823e+20,Alvin Martinez,2020-10-23,5,I'm late to posting this but this store especi...,sin respuesta,NaT
1,2,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.0729344149210933e+20,Johnnie Jackson,2021-05-04,1,Very dissatisfied I did not get my phone the p...,We pride ourselves on providing an awesome exp...,2021-05-06
2,3,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.003785858018194e+20,Manie Blazer,2020-08-14,5,Excellent very well done with professional car...,sin respuesta,NaT
3,4,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.1499816115301982e+20,Fashion Fiinds,2018-12-02,5,Basing my review strictly on the service I rec...,Thanks for the awesome review! We work hard t...,2018-12-03
4,5,0x89c25fc9494dce47:0x6d63c807b59a55,New York,1.171781857284223e+20,Andres Rieloff,2020-08-13,1,Bad! Disorganized. I'm being totally honest. I...,sin respuesta,NaT
...,...,...,...,...,...,...,...,...,...,...
2296819,12046820,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0216813568763861e+20,t williams,2019-07-11,5,sin comentario,sin respuesta,NaT
2296820,12046821,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0541730918471213e+20,Erika Lira,2018-07-19,5,sin comentario,sin respuesta,NaT
2296821,12046822,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.10785699186536e+20,Jon Brent,2019-09-08,5,sin comentario,sin respuesta,NaT
2296822,12046823,0x8640d215ff1e43e5:0xaeeb5211a25191f4,Texas,1.0916760195743087e+20,lunna cabal,2017-09-27,4,sin comentario,sin respuesta,NaT


In [106]:
# Verifico nulos
nulos_por_columna = g_estados_seleccionados.isnull().sum()
print(nulos_por_columna)

review_id            0
id_negocio           0
estado               0
user_id              0
name                 0
time                 0
rating               0
text                 0
resp_text            0
resp_time     11307310
dtype: int64


Los nulos en la columna resp_time se dejaran así para poder almacenarse luego el dataframe en formato parquet. En un futuro análisis para su utilización se trataran dichos nulos.

Voy a importar el dataframe v1.g_sitios con el objetivo de filtrar el dataframe creado recién por las categorías elegidas para este proyecto. 

In [107]:
sitios= pd.read_parquet(r"C:\Users\Juli\Desktop\Data Science\Archivos Proyecto Final\v1.g_sitios.parquet")
sitios.head()

,name,id_negocio,avg_rating,num_of_reviews,url,monday,tuesday,wednesday,thursday,friday,saturday,sunday,category_general_id,id_estado
0,Oneyda's Bakery,0x88dae191ee505917:0x6ba3e25388d3fad4,4.6,19,https://www.google.com/maps/place//data=!4m2!3...,8AM–6PM,8AM–6PM,8AM–6PM,8AM–6PM,8AM–6PM,8AM–6PM,Closed,4,7
1,TACOS LA CABANA,0x808f879f35b5088b:0xe3541cec7a95bd88,5.0,2,https://www.google.com/maps/place//data=!4m2!3...,5–11PM,Closed,Closed,Closed,5–11PM,5–11PM,5–11PM,3,9
2,Tropical Park Liquors,0x88d9b99475d9fd7b:0xea6083d207b2471a,4.7,8,https://www.google.com/maps/place//data=!4m2!3...,10AM–10PM,10AM–10PM,10AM–12AM,10AM–12AM,10AM–12AM,10AM–12AM,10AM–10PM,4,7
3,The Nutrition Group,0x89c88de475520cc7:0xeff46469445b5212,3.2,17,https://www.google.com/maps/place//data=!4m2!3...,9AM–8PM,9AM–8PM,9AM–8PM,9AM–8PM,9AM–8PM,9AM–5PM,Closed,4,13
4,Top Cat Seafood Restaurant,0x864e9891e381f3df:0x4cefe6219bc9199c,3.9,8,https://www.google.com/maps/place//data=!4m2!3...,12–8PM,12–8PM,12–8PM,12–8PM,12–8PM,12–8PM,12–8PM,3,18


In [108]:
# Realizamos el merge para obtener solo las filas de g_estados_seleccionados que tienen coincidencia en sitios
tabla_final = pd.merge(g_estados_seleccionados, sitios, on='id_negocio', how='inner')
tabla_final

,review_id,id_negocio,estado,user_id,name_x,time,rating,text,resp_text,resp_time,...,url,monday,tuesday,wednesday,thursday,friday,saturday,sunday,category_general_id,id_estado
0,148,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.018634613890113e+20,Maria Patricia Londoño,2021-08-16,4,The donuts is always a good place to buy somet...,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,4,24
1,149,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0195463333253585e+20,Kristal,2020-07-01,5,I went into this Dunkin' yesterday and got som...,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,4,24
2,150,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0898768409594107e+20,Efrain Hernandez,2020-07-07,3,Bought a Machiato and as soon as I touched the...,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,4,24
3,151,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1014632117097557e+20,Kiyoshi Sudo,2019-08-02,5,"Friendly staffs, nice donuts and muffins and c...",sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,4,24
4,152,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1262588515936902e+20,Charlotte Sheppard,2020-10-11,1,They got my order wrong food wasn't done unco...,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,6AM–7PM,4,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4128016,12046147,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.086757371066782e+20,Gary Kohr,2014-08-10,2,Maybe it was bad because it's Sunday but only ...,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–10PM,11AM–10PM,11AM–9:30PM,3,18
4128017,12046148,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0378064778394966e+20,Joyce Earls,2016-09-23,5,sin comentario,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–10PM,11AM–10PM,11AM–9:30PM,3,18
4128018,12046149,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.1824656293609538e+20,Brenda Leffler,2012-09-28,3,sin comentario,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–10PM,11AM–10PM,11AM–9:30PM,3,18
4128019,12046150,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0440035146720169e+20,Andrés Peña,2017-06-07,1,sin comentario,sin respuesta,NaT,...,https://www.google.com/maps/place//data=!4m2!3...,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–9:30PM,11AM–10PM,11AM–10PM,11AM–9:30PM,3,18


In [109]:
# Seleccionamos solo las columnas específicas del dataframe resultado
tabla_final = tabla_final[['review_id', 'id_negocio', 'estado', 'user_id', 'name_x', 'time', 'rating', 'text', 'resp_text', 'resp_time']]
tabla_final

,review_id,id_negocio,estado,user_id,name_x,time,rating,text,resp_text,resp_time
0,148,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.018634613890113e+20,Maria Patricia Londoño,2021-08-16,4,The donuts is always a good place to buy somet...,sin respuesta,NaT
1,149,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0195463333253585e+20,Kristal,2020-07-01,5,I went into this Dunkin' yesterday and got som...,sin respuesta,NaT
2,150,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0898768409594107e+20,Efrain Hernandez,2020-07-07,3,Bought a Machiato and as soon as I touched the...,sin respuesta,NaT
3,151,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1014632117097557e+20,Kiyoshi Sudo,2019-08-02,5,"Friendly staffs, nice donuts and muffins and c...",sin respuesta,NaT
4,152,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1262588515936902e+20,Charlotte Sheppard,2020-10-11,1,They got my order wrong food wasn't done unco...,sin respuesta,NaT
...,...,...,...,...,...,...,...,...,...,...
4128016,12046147,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.086757371066782e+20,Gary Kohr,2014-08-10,2,Maybe it was bad because it's Sunday but only ...,sin respuesta,NaT
4128017,12046148,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0378064778394966e+20,Joyce Earls,2016-09-23,5,sin comentario,sin respuesta,NaT
4128018,12046149,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.1824656293609538e+20,Brenda Leffler,2012-09-28,3,sin comentario,sin respuesta,NaT
4128019,12046150,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0440035146720169e+20,Andrés Peña,2017-06-07,1,sin comentario,sin respuesta,NaT


In [110]:
# Cambiamos el nombre de la columna 'name_x' a 'name'
tabla_final = tabla_final.rename(columns={'name_x': 'name'})
tabla_final

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,148,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.018634613890113e+20,Maria Patricia Londoño,2021-08-16,4,The donuts is always a good place to buy somet...,sin respuesta,NaT
1,149,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0195463333253585e+20,Kristal,2020-07-01,5,I went into this Dunkin' yesterday and got som...,sin respuesta,NaT
2,150,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0898768409594107e+20,Efrain Hernandez,2020-07-07,3,Bought a Machiato and as soon as I touched the...,sin respuesta,NaT
3,151,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1014632117097557e+20,Kiyoshi Sudo,2019-08-02,5,"Friendly staffs, nice donuts and muffins and c...",sin respuesta,NaT
4,152,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1262588515936902e+20,Charlotte Sheppard,2020-10-11,1,They got my order wrong food wasn't done unco...,sin respuesta,NaT
...,...,...,...,...,...,...,...,...,...,...
4128016,12046147,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.086757371066782e+20,Gary Kohr,2014-08-10,2,Maybe it was bad because it's Sunday but only ...,sin respuesta,NaT
4128017,12046148,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0378064778394966e+20,Joyce Earls,2016-09-23,5,sin comentario,sin respuesta,NaT
4128018,12046149,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.1824656293609538e+20,Brenda Leffler,2012-09-28,3,sin comentario,sin respuesta,NaT
4128019,12046150,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0440035146720169e+20,Andrés Peña,2017-06-07,1,sin comentario,sin respuesta,NaT


In [111]:
# Guardamos el dataframe en un archivo parquet
tabla_final.to_parquet(r"C:\Users\Juli\Desktop\v0_estados_reviews.parquet")

In [2]:
import pandas as pd
reviews = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\v0_estados_reviews.parquet', engine='pyarrow')
reviews

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
0,148,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.018634613890113e+20,Maria Patricia Londoño,2021-08-16,4,The donuts is always a good place to buy somet...,sin respuesta,NaT
1,149,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0195463333253585e+20,Kristal,2020-07-01,5,I went into this Dunkin' yesterday and got som...,sin respuesta,NaT
2,150,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0898768409594107e+20,Efrain Hernandez,2020-07-07,3,Bought a Machiato and as soon as I touched the...,sin respuesta,NaT
3,151,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1014632117097557e+20,Kiyoshi Sudo,2019-08-02,5,"Friendly staffs, nice donuts and muffins and c...",sin respuesta,NaT
4,152,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1262588515936902e+20,Charlotte Sheppard,2020-10-11,1,They got my order wrong food wasn't done unco...,sin respuesta,NaT
...,...,...,...,...,...,...,...,...,...,...
4128016,12046147,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.086757371066782e+20,Gary Kohr,2014-08-10,2,Maybe it was bad because it's Sunday but only ...,sin respuesta,NaT
4128017,12046148,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0378064778394966e+20,Joyce Earls,2016-09-23,5,sin comentario,sin respuesta,NaT
4128018,12046149,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.1824656293609538e+20,Brenda Leffler,2012-09-28,3,sin comentario,sin respuesta,NaT
4128019,12046150,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0440035146720169e+20,Andrés Peña,2017-06-07,1,sin comentario,sin respuesta,NaT


In [ ]:
import sys
sys.path.append(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\src')
from analisis_sentimiento.sentimiento import analyze_sentiment

reviews['sentiment'] = reviews.apply(
    lambda row: analyze_sentiment(row['text'], rating=row.get('rating', None)), 
    axis=1
)


In [38]:
reviews['sentiment'] = reviews.apply(lambda row: 'sin comentario' if row['text'] == 'sin comentario' else row['sentiment'], axis=1)
reviews

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time,sentiment
0,148,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.018634613890113e+20,Maria Patricia Londoño,2021-08-16,4,The donuts is always a good place to buy somet...,sin respuesta,NaT,bueno
1,149,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0195463333253585e+20,Kristal,2020-07-01,5,I went into this Dunkin' yesterday and got som...,sin respuesta,NaT,bueno
2,150,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0898768409594107e+20,Efrain Hernandez,2020-07-07,3,Bought a Machiato and as soon as I touched the...,sin respuesta,NaT,neutro
3,151,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1014632117097557e+20,Kiyoshi Sudo,2019-08-02,5,"Friendly staffs, nice donuts and muffins and c...",sin respuesta,NaT,bueno
4,152,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1262588515936902e+20,Charlotte Sheppard,2020-10-11,1,They got my order wrong food wasn't done unco...,sin respuesta,NaT,malo
...,...,...,...,...,...,...,...,...,...,...,...
4128016,12046147,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.086757371066782e+20,Gary Kohr,2014-08-10,2,Maybe it was bad because it's Sunday but only ...,sin respuesta,NaT,neutro
4128017,12046148,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0378064778394966e+20,Joyce Earls,2016-09-23,5,sin comentario,sin respuesta,NaT,sin comentario
4128018,12046149,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.1824656293609538e+20,Brenda Leffler,2012-09-28,3,sin comentario,sin respuesta,NaT,sin comentario
4128019,12046150,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0440035146720169e+20,Andrés Peña,2017-06-07,1,sin comentario,sin respuesta,NaT,sin comentario


In [1]:
import pandas as pd
reviews = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\v0_g_reviews.parquet', engine='pyarrow')
reviews

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time,sentiment
0,148,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.018634613890113e+20,Maria Patricia Londoño,2021-08-16,4,The donuts is always a good place to buy somet...,sin respuesta,NaT,bueno
1,149,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0195463333253585e+20,Kristal,2020-07-01,5,I went into this Dunkin' yesterday and got som...,sin respuesta,NaT,bueno
2,150,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.0898768409594107e+20,Efrain Hernandez,2020-07-07,3,Bought a Machiato and as soon as I touched the...,sin respuesta,NaT,neutro
3,151,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1014632117097557e+20,Kiyoshi Sudo,2019-08-02,5,"Friendly staffs, nice donuts and muffins and c...",sin respuesta,NaT,bueno
4,152,0x89c261f60bdf13db:0x38da730e4687a97b,New York,1.1262588515936902e+20,Charlotte Sheppard,2020-10-11,1,They got my order wrong food wasn't done unco...,sin respuesta,NaT,malo
...,...,...,...,...,...,...,...,...,...,...,...
4128016,12046147,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.086757371066782e+20,Gary Kohr,2014-08-10,2,Maybe it was bad because it's Sunday but only ...,sin respuesta,NaT,neutro
4128017,12046148,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0378064778394966e+20,Joyce Earls,2016-09-23,5,sin comentario,sin respuesta,NaT,sin comentario
4128018,12046149,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.1824656293609538e+20,Brenda Leffler,2012-09-28,3,sin comentario,sin respuesta,NaT,sin comentario
4128019,12046150,0x863f79137a27e1e7:0x5e916e7fe74dcff5,Texas,1.0440035146720169e+20,Andrés Peña,2017-06-07,1,sin comentario,sin respuesta,NaT,sin comentario


In [5]:
filtro = reviews[reviews['time'] >= '2019-01-01']
filtro = filtro.drop(columns=['user_id', 'name', 'resp_text', 'resp_time'])
filtro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2380094 entries, 0 to 4127988
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   time        datetime64[ns]
 4   rating      int64         
 5   text        object        
 6   sentiment   object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 145.3+ MB


In [6]:
filtro.to_parquet('bi_g_reviews.parquet', engine='pyarrow')